In [135]:
import pandas as pd

In [136]:
df = pd.read_csv('/content/drive/MyDrive/salary prediction/data_salary.csv')

In [137]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [138]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,373.0,37.431635,7.069073,23.0,31.0,36.0,44.0,53.0
Years of Experience,373.0,10.030831,6.557007,0.0,4.0,9.0,15.0,25.0
Salary,373.0,100577.345845,48240.013482,350.0,55000.0,95000.0,140000.0,250000.0


In [139]:
df.shape

(375, 6)

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  373 non-null    float64
 1   Gender               373 non-null    object 
 2   Education Level      373 non-null    object 
 3   Job Title            373 non-null    object 
 4   Years of Experience  373 non-null    float64
 5   Salary               373 non-null    float64
dtypes: float64(3), object(3)
memory usage: 17.7+ KB


In [141]:
df.isnull().sum().sum()

12

In [142]:
df.isnull().sum()

,0
Age,2
Gender,2
Education Level,2
Job Title,2
Years of Experience,2
Salary,2


In [143]:
df.dropna(axis = 0 , inplace =True)

In [144]:
df.isnull().sum()

,0
Age,0
Gender,0
Education Level,0
Job Title,0
Years of Experience,0
Salary,0


In [145]:
df.duplicated().sum()

49

In [146]:
df.drop_duplicates(inplace = True)

In [147]:
df.duplicated().sum()

0

In [149]:
df.shape

(324, 6)

In [150]:
df.columns

Index(['Age', 'Gender', 'Education Level', 'Job Title', 'Years of Experience',
       'Salary'],
      dtype='object')

In [158]:
x = df[['Age', 'Gender', 'Education Level', 'Years of Experience']]
y = df[['Salary']]

In [171]:
### encoding categorical data:
import numpy as np
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder , LabelEncoder , StandardScaler
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

gender_feature = ['Gender']
gender_transformer = Pipeline(
    steps =[('encoder', OneHotEncoder(handle_unknown = 'ignore')),
            ('selector' , SelectPercentile(chi2 , percentile =25))
            ]
)


edu_feature = ['Education Level']
edu_transformer = Pipeline(
    steps = [
        ("encoder" , OrdinalEncoder(handle_unknown = 'error')),
        ("selector" , SelectPercentile(chi2, percentile=50))
    ]
)



numeric_features = x.select_dtypes(include = 'number').columns.to_list()
numeric_transformer = Pipeline(
    steps = [('imputer' , SimpleImputer(strategy ='median')),('scaler',StandardScaler())]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('numerical' , numeric_transformer , numeric_features),
        ('gender' , gender_transformer , gender_feature),
        ('education' , edu_transformer , edu_feature)
    ]
)

clf = Pipeline(
    steps=[
        ('processor' , preprocessor),
        ('regression' , LinearRegression())
           ]
)



In [172]:
X_train, X_test, y_train, y_test = train_test_split(x , y , test_size=0.2 ,random_state=0)


In [173]:
clf.fit(X_train , y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_base.py:102: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


Pipeline(steps=[('processor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age',
                                                   'Years of Experience']),
                                                 ('gender',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=25,
                                                                                    score_func=<function chi2 at 0x7a2fd5fc4ee0>))]),
                                                  ['Gender']),
                                                 ('education',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder()),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x7a2fd5fc4ee0>))]),
                                                  ['Education Level'])])),
                ('regression', LinearRegression())])

In [174]:
from sklearn.metrics import r2_score

#R squared for training
y_pred_train = clf.predict(X_train)
print("R squared for train: %.2f" % r2_score(y_train, y_pred_train))


#R squared for testing
y_pred_test = clf.predict(X_test)
print("R squared for test: %.2f" % r2_score(y_test, y_pred_test))

R squared for train: 0.87
R squared for test: 0.84


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_base.py:102: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_base.py:102: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
